<a href="https://colab.research.google.com/github/mateusduarte-max/Projetos_Python/blob/main/05_Ciclo_Pedido_Nets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install --upgrade openpyxl # atualização pacote excel
!pip install workadays # modulo para calcular data prometida e data limite de postagem
!pip install unidecode # modulo para retirar caracteres especiais

import pandas as pd
import re
import datetime
from workadays import workdays as wd
import numpy as np
import unidecode
from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
from gspread_dataframe import set_with_dataframe
from google.colab import drive
drive.mount('/content/drive')
# Modulos 
!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/modulos_colab.py /content
!cp /content/drive/MyDrive/Colab_Notebooks/Autenticacoes/chaves_tokens.py /content
%run chaves_tokens.py
# Importar biblioteca biqquery
from modulos_colab import dados


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 29.3 MB/s 
Mounted at /content/drive


Ciclo Pedido Nets

In [ ]:
# Variaveis Globais
data_base = (datetime.datetime.now() - datetime.timedelta(45)).strftime('%Y-%m-%d')

# Consulta reclamações formais

reclamacoes_formais = dados.consulta_bigquery(projeto_gcp_1,"""SELECT W101.* ,
                                                                        cast(a.PEDS_DAT_CAD as date) as Data_captado,cast(a.PEDS_DAT_FAT as date) as Data_faturado,cast(a.PEDS_DAT_PRZINI as date) as Data_Prazo_Entrega, tetrix.* except(code),a.PEDS_COD ,a.PEDS_EXT_COD,

                                                                        CASE 
                                                                            WHEN UPPER(CAST(a.PEDS_COD as string)) like '%N%'  THEN 'Pedido Reverso'
                                                                            WHEN UPPER(CAST(a.PEDS_EXT_COD  as string)) like '%N%' then 'Pedido Reverso'
                                                                            WHEN UPPER(CAST(a.PEDS_COD as string)) like '%Z%'  THEN 'Pedido Reverso' 
                                                                            WHEN UPPER(CAST(a.PEDS_EXT_COD  as string)) like '%Z%' then 'Pedido Reverso'     
                                                                        ELSE 'Não é Reverso' end as Pedido_Reverso, 

                                                                        CASE
                                                                        WHEN W101.LojaWebNome in ('ALLIANZ PARQUE','B2b Coorporativo','B2b Netshoes','CLUBE NETSHOES','Csu','CSU Passaporte Opte+','DIGI NETSHOES','Dotz','Freedom - Chapecoense','Freedom - Freelace','Freedom - Kappa','Freedom - Loja Cruzeiro','Freedom - Loja Internacional','Freedom - Loja Nba','Freedom - Netshoes','Freedom - Palmeiras','Freedom - Santos','Freedom - Sao Paulo','Freedom - Shoptimao','Freedom - Vasco','Func Netshoes','ITAU IUPP NETSHOES','Livelo','LOJA GAP','LOJA NFL','LOJA WSL','Ltm','MARKUP NETSHOES','Multiplus','Nba','NETSHOES','NIKE FUNCIONARIOS','NS2 - AMZ (Amazon)','NS2 - B2W','NS2 - CRF (Carrefour)','NS2 - MGL (Magazine Luiza)','NS2 - MLI (Mercado Livre)','NS2 - VVJ (ViaVarejo)','Shoestock','Shoestock Loja Fisica','SMILES NETSHOES','TUDO AZUL - NETSHOES','B2W','CRUZEIRO','Csu Banescard','Csu Banestes','Csu Banpará','Csu Banrisul','Csu Electrolux','Csu Pernambucanas','Csu Porto Seguro','Csu Vivo','Loja Do Itaú','MULTIPLUS NETSHOES MKTP','NS2 - BCP','PALMEIRAS','SANTOS','SÃO PAULO MANIA','SHOPTIMAO','Vasco') THEN 'Nets'
                                                                        WHEN W101.LojaWebNome in ('CSU - SHOPPING ZATTINI','Csu Banrisul - Zattini','Csu Passaporte Opte+ - Zattini','Freedom - Zattini','Livelo Zattini','Ltm Zattini','MARKUP ZATTINI','Multiplus Zattini','SHOPFACIL ZATTINI','Smiles Zattini','TUDO AZUL - ZATTINI','ZATTINI','Csu Banescard - Zattini','Csu Banestes - Zattini','Csu Banpará - Zattini','Csu Electrolux - Zattini','Csu Pernambucanas - Zattini','Csu Porto Seguro - Zattini','Csu Vivo - Zattini','MULTIPLUS ZATTINI MKTP') THEN 'Zattini'
                                                                        ELSE W101.LojaWebNome END as Depara_Grupo,

                                                                        CASE 
                                                                        WHEN W101.LojaWebNome in ('ALLIANZ PARQUE') THEN 'ALLIANZ PARQUE'
                                                                        WHEN W101.LojaWebNome in ('Freedom - Chapecoense') THEN 'Chapecoense' 
                                                                        WHEN W101.LojaWebNome in ('Freedom - Loja Cruzeiro','CRUZEIRO') THEN 'Cruzeiro' 
                                                                        WHEN W101.LojaWebNome in ('Freedom - Freelace') THEN 'Freelace' 
                                                                        WHEN W101.LojaWebNome in ('Func Netshoes') THEN 'Func Netshoes' 
                                                                        WHEN W101.LojaWebNome in ('LOJA GAP') THEN 'GAP' 
                                                                        WHEN W101.LojaWebNome in ('Freedom - Loja Internacional') THEN 'Internacional' 
                                                                        WHEN W101.LojaWebNome in ('ITAU IUPP NETSHOES','Loja Do Itaú') THEN 'IUPP' 
                                                                        WHEN W101.LojaWebNome in ('Freedom - Kappa') THEN 'Kappa' 
                                                                        WHEN W101.LojaWebNome in ('Freedom - Loja Nba','Nba') THEN 'Nba' 
                                                                        WHEN W101.LojaWebNome in ('LOJA NFL') THEN 'NFL' 
                                                                        WHEN W101.LojaWebNome in ('NIKE FUNCIONARIOS') THEN 'NIKE FUNCIONARIOS' 
                                                                        WHEN W101.LojaWebNome in ('Freedom - Palmeiras','PALMEIRAS') THEN 'Palmeiras' 
                                                                        WHEN W101.LojaWebNome in ('Freedom - Santos','SANTOS') THEN 'Santos' 
                                                                        WHEN W101.LojaWebNome in ('Freedom - Sao Paulo','SÃO PAULO MANIA') THEN 'Sao Paulo' 
                                                                        WHEN W101.LojaWebNome in ('Shoestock Loja Fisica','Shoestock') THEN 'Shoestock' 
                                                                        WHEN W101.LojaWebNome in ('Freedom - Shoptimao','SHOPTIMAO') THEN 'Shoptimao' 
                                                                        WHEN W101.LojaWebNome in ('Freedom - Vasco','Vasco') THEN 'Vasco' 
                                                                        WHEN W101.LojaWebNome in ('LOJA WSL') THEN 'WSL' 
                                                                        WHEN W101.LojaWebNome in ('CSU - SHOPPING ZATTINI','Csu Banrisul - Zattini','Csu Passaporte Opte+ - Zattini','Freedom - Zattini','Livelo Zattini','Ltm Zattini','MARKUP ZATTINI','Multiplus Zattini','SHOPFACIL ZATTINI','Smiles Zattini','TUDO AZUL - ZATTINI','ZATTINI','Csu Banescard - Zattini','Csu Banestes - Zattini','Csu Banpará - Zattini','Csu Electrolux - Zattini','Csu Pernambucanas - Zattini','Csu Porto Seguro - Zattini','Csu Vivo - Zattini','MULTIPLUS ZATTINI MKTP') 
                                                                        THEN 'Zattini' 
                                                                        WHEN LojaWebNome in ('B2b Coorporativo','B2b Netshoes','CLUBE NETSHOES','Csu','CSU Passaporte Opte+','DIGI NETSHOES','Dotz','Freedom - Chapecoense','Freedom - Freelace','Freedom - Kappa','Freedom - Loja Cruzeiro','Freedom - Loja Internacional','Freedom - Loja Nba','Freedom - Netshoes','Freedom - Palmeiras','Freedom - Santos','Freedom - Sao Paulo','Freedom - Shoptimao','Freedom - Vasco','Func Netshoes','ITAU IUPP NETSHOES','Livelo','LOJA GAP','LOJA NFL','LOJA WSL','Ltm','MARKUP NETSHOES','Multiplus','Nba','NETSHOES','NIKE FUNCIONARIOS','NS2 - AMZ (Amazon)','NS2 - B2W','NS2 - CRF (Carrefour)','NS2 - MGL (Magazine Luiza)','NS2 - MLI (Mercado Livre)','NS2 - VVJ (ViaVarejo)','Shoestock','Shoestock Loja Fisica','SMILES NETSHOES','TUDO AZUL - NETSHOES','B2W','CRUZEIRO','Csu Banescard','Csu Banestes','Csu Banpará','Csu Banrisul','Csu Electrolux','Csu Pernambucanas','Csu Porto Seguro','Csu Vivo','Loja Do Itaú','MULTIPLUS NETSHOES MKTP','NS2 - BCP') THEN 'Nets' 
                                                                        ELSE LojaWebNome END as Depara_Loja,

                                                                        CASE 
                                                                        WHEN lower(W101.Transportadora) in ('air cargo recife ltda') THEN 'Air Cargo LTDA'
                                                                        WHEN lower(W101.Transportadora) in ('atlas transportes') THEN 'Atlas Transprote'
                                                                        WHEN lower(W101.Transportadora) in ('carriers logistica e transportes ltda me') THEN 'Carriers Logística'
                                                                        WHEN lower(W101.Transportadora) in ('empresa brasileira de correios e telegrafos') THEN 'Correios'
                                                                        WHEN lower(W101.Transportadora) in ('dgt logistica e transportes ltda.','dgt logística e transportes ltda.') THEN 'DGT Logística'
                                                                        WHEN lower(W101.Transportadora) in ('dialogo logistica inteligente ltda epp','diálogo logística inteligente ltda epp') THEN 'Díalogo Logística'
                                                                        WHEN lower(W101.Transportadora) in ('direct express logistica integrada s/a','direct express logística integrada s/a') THEN 'Direct Express'
                                                                        WHEN lower(W101.Transportadora) in ('dsl negocios e servicos especiais ltda','dsl negócios e serviços especiais ltda') THEN 'DSL Logística'
                                                                        WHEN lower(W101.Transportadora) in ('ecolivery courieros me') THEN 'Ecolivery Logística'
                                                                        WHEN lower(W101.Transportadora) in ('expresso jundiai logistica e transporte ltda') THEN 'Expresso Logística'
                                                                        WHEN lower(W101.Transportadora) in ("faro's - transportes comercio importac?o e exportac?o ltda","faro's - transportes comércio importação e exportação ltda") THEN 'Faro Transportes'
                                                                        WHEN lower(W101.Transportadora) in ('fast courier servicos ltda','fast courier serviços ltda') THEN 'Fast Courier LTDA'
                                                                        WHEN lower(W101.Transportadora) in ('gfl logistica ltda') THEN 'GFL Logística'
                                                                        WHEN lower(W101.Transportadora) in ('inside the time transportes ltda.') THEN 'Inside the Time LTDA'
                                                                        WHEN lower(W101.Transportadora) in ('jadlog logistica ltda','jadlog logistica ltda.') THEN 'Jadlog'
                                                                        WHEN lower(W101.Transportadora) in ('jamef transportes') THEN 'Jamef Transportes'
                                                                        WHEN lower(W101.Transportadora) in ('l4b logistica ltda') THEN 'L4B Logística'
                                                                        WHEN lower(W101.Transportadora) in ('lmtrans logistica em transportes ltda') THEN 'LMTrans Logística'
                                                                        WHEN lower(W101.Transportadora) in ('logbee','expressa- logbee') THEN 'Logbee'
                                                                        WHEN lower(W101.Transportadora) in ('magalog') THEN 'Magalog'
                                                                        WHEN lower(W101.Transportadora) in ('nhl logistica inteligente ltda') THEN 'NHL Logística'
                                                                        WHEN lower(W101.Transportadora) in ('on time express logistica e transportes ltda') THEN 'On time'
                                                                        WHEN lower(W101.Transportadora) in ('rapid?o cometa logistica e transporte s/a','rapidão cometa logistica e transporte s/a','rapidão cometa logística e transporte s/a') THEN 'Rapidão Cometa'
                                                                        WHEN lower(W101.Transportadora) in ('rbs - zero hora editora jornalistica s.a.') THEN 'RBS Transportes'
                                                                        WHEN lower(W101.Transportadora) in ('sem transportadora') THEN 'Sem Transportadora'
                                                                        WHEN lower(W101.Transportadora) in ('sequoia logistica e transportes sa','sequoia logística e transportes sa') THEN 'Sequoia Logística'
                                                                        WHEN lower(W101.Transportadora) in ('total express') THEN 'Total Express'
                                                                        WHEN lower(W101.Transportadora) in ('transfolha transporte e distribuic?o ltda','transfolha transporte e distribuicão ltda','transfolha transporte e distribuição ltda') THEN 'Transfolha'
                                                                        WHEN lower(W101.Transportadora) in ('transportadora netshoes','netshoes - frota propria') THEN 'Transportadora Netshoes'
                                                                        WHEN lower(W101.Transportadora) in ('air express fermac assessoria e consultoria em logistica e transporte ltda') THEN 'Air Express Logística'
                                                                        WHEN lower(W101.Transportadora) in ('braspress transportes urgentes ltda') THEN 'Braspress Transportes'
                                                                        WHEN lower(W101.Transportadora) in ('cajumar transportes ltda') THEN 'Cajumar Transportes'
                                                                        WHEN lower(W101.Transportadora) in ('ctl logistica me') THEN 'CTL Logística'
                                                                        WHEN lower(W101.Transportadora) in ('emerson rodrigo violin me') THEN 'Emerson Rodrigo Violin ME'
                                                                        WHEN lower(W101.Transportadora) in ('gdr express network transportes ltda') THEN 'GDR Express'
                                                                        WHEN lower(W101.Transportadora) in ('gps motos') THEN 'GPS Motos'
                                                                        WHEN lower(W101.Transportadora) in ('irmãos silveira transportes em geral ltda') THEN 'Silveira Transportes'
                                                                        WHEN lower(W101.Transportadora) in ('leonardo zenko pires de carvalho - me') THEN 'LEONARDO ZENKO PIRES DE CARVALHO - ME'
                                                                        WHEN lower(W101.Transportadora) in ('lmtrans logística em transportes ltda') THEN 'LMTRANS Logística'
                                                                        WHEN lower(W101.Transportadora) in ('movmov tecnologia da informacao ltda') THEN 'MovMov Tecnologias'
                                                                        WHEN lower(W101.Transportadora) in ('nfn ferreira logistica em transportes de carga ltda') THEN 'NFN Logística'
                                                                        WHEN lower(W101.Transportadora) in ('pacifico log transportes') THEN 'Pacífico Transportes'
                                                                        WHEN lower(W101.Transportadora) in ('ramos transportes') THEN 'Ramos Transportes'
                                                                        WHEN lower(W101.Transportadora) in ('ss reversa e projetos customizados ltda - epp') THEN 'SS Reversa'
                                                                        WHEN lower(W101.Transportadora) in ('tex log') THEN 'Tex Log'
                                                                        WHEN lower(W101.Transportadora) in ('transp. padrao mkp') THEN 'Transportadora Padrão MKP'
                                                                        WHEN lower(W101.Transportadora) in ('transportadora route ltda') THEN 'Route Transportadora'
                                                                        WHEN lower(W101.Transportadora) in ('tsv transportes rapidos') THEN 'TSV Transportes'
                                                                        WHEN lower(W101.Transportadora) in ('transportadora shoestock') THEN 'Transportadora Shoestock'
                                                                        WHEN lower(W101.Transportadora) in ('wf gomes transportes - me') THEN 'WF Transportes'
                                                                        WHEN lower(W101.Transportadora) in ('empresa brasileira de transportes') THEN 'Empresa Brasileira de Transportes'
                                                                        ELSE lower(W101.Transportadora) end as Depara_Transportadora,

                                                                        FROM `maga-bigdata.sac.pc_abacos_sac` AS W101
                                                                        LEFT JOIN `maga-bigdata.abacos_nets.aba_tcom_pedsai` as a on a.PEDS_COD = W101.Pedido
                                                                        LEFT JOIN (SELECT DISTINCT o.code, ifnull(STRING_AGG(DISTINCT b.name,"/" order by b.name),'Pedido_1P') as seller_id_pedido, ifnulL(STRING_AGG(DISTINCT c.distribution_center_name,"/" order by c.distribution_center_name),'Sem CD Cadastrado') as cds_pedido,
                                                                                MIN(billing_date) as dt_pagamento_menor_pedido,MIN(dispatch_date) as dt_despacho_menor_pedido,MAX(o.sales_channel) as canal_venda
                                                                                FROM `maga-bigdata.tetrix.vw_order_sale` as o
                                                                                LEFT JOIN `maga-bigdata.tetrix.vw_order_shipping` as a on a.order_id = o.id
                                                                                LEFT JOIN `maga-bigdata.tetrix.vw_seller` as b on cast(b.code as string) = a.seller_id
                                                                                LEFT JOIN `maga-bigdata.tetrix.vw_shipping_product` as c on c.order_shipping_id = a.id
                                                                                GROUP BY 1)as tetrix on tetrix.code = a.PEDS_EXT_COD 
                                                                        WHERE 1=1
                                                                        AND AtendimentoOrigemNome in ('Jurídico Procon','Jurídico Cível')
                                                                        AND W101.AtendimentoServicoNome  in ('Reclamações formais')
                                                                        AND CAST(AtendimentoDataInicial as date) >= '{}'""".format(data_base))

# Tratativa base
reclamacoes_formais.rename(columns={'seller_id_pedido':'LOJISTA'}, inplace=True)
reclamacoes_formais['LOJISTA'] = reclamacoes_formais['LOJISTA'].str.strip().str.lower().str.split('/',1).str[0].str.split('-', 1).str[0]
reclamacoes_formais_resumo = pd.pivot_table(reclamacoes_formais, index='LOJISTA', values='AtendimentoCodigo', aggfunc='count', margins=True)
reclamacoes_formais_resumo.rename(columns={'AtendimentoCodigo':'Qtd_Reclamacoes_Formais'}, inplace=True)

# Grava na tabela (Google Drive)
 
#dados.grava_tabela_drive('Base_reclamacoes_formais', reclamacoes_formais)
print("Bloco executado")

#reclamacoes_formais = busca_tabela_drive('Base_reclamacoes_formais')
#reclamacoes_formais.rename(columns={'seller_id_pedido':'LOJISTA'}, inplace=True)
#reclamacoes_formais['LOJISTA'] = reclamacoes_formais['LOJISTA'].str.strip().str.lower().str.split('/',1).str[0].str.split('-', 1).str[0]
#reclamacoes_formais_resumo = pd.pivot_table(reclamacoes_formais, index='LOJISTA', values='AtendimentoCodigo', aggfunc='count', margins=True)
#reclamacoes_formais_resumo.rename(columns={'AtendimentoCodigo':'Qtd_Reclamacoes_Formais'}, inplace=True)

#########################################################################################################################################################################################

# Dados pedidos com checkin
'''pedidos_ckeckin = dados.consulta_bigquery(projeto_gcp_1,"""SELECT DISTINCT
                                                            A.PEDS_COD COD_PEDIDO,
                                                            --A.PEDS_EXT_COD PEDIDO_EXTERNO,
                                                            --B.TROD_COD CODIGO_DA_TROCA,
                                                            B.TROD_STR_STA STATUS_checkin,
                                                            --C.ITET_DAT_PROENT DATA_CHECKIN
                                                            FROM `maga-bigdata.abacos_nets.aba_tcom_pedsai` A
                                                            INNER JOIN `maga-bigdata.abacos_nets.aba_ttrd_trodev` B ON A.CLIF_COD = B.CLIF_COD
                                                            --INNER JOIN `maga-bigdata.abacos_nets.aba_ttrd_itetrd` C ON B.TROD_COD = C.TROD_COD
                                                            --WHERE A.PEDS_COD IN ('XXX')
                                                            WHERE B.TROD_STR_STA in ('Finalizado', 'Aberto')	
                                                                --AND PEDS_VAL_PED = ITET_VAL_PRETOT
                                                                --ITET_DAT_PROENT IS NOT NULL
                                                                --ITET_DAT_PROENT >='2019-01-01'
                                                                --AND B.TROD_DAT_PRAPRO >= '2021-09-01'
                                                                --and A.PEDS_COD = 864944870
                                                            group by A.PEDS_COD, B.TROD_STR_STA""")


pedidos_ckeckin['COD_PEDIDO'] = pedidos_ckeckin['COD_PEDIDO'].astype(str)'''
###################### Consulta pedidos nets bq #######################################################################################################
pedidos = dados.consulta_bigquery(projeto_gcp_1,"""Select distinct
                                                    shp.seller_name as LOJISTA,
                                                    shp.seller_id as ID_SELLER,
                                                    os.code as COD_PEDIDO_LOJA,
                                                    osh.delivery_id as COD_PEDIDO,
                                                    cast(dispatch_date as date) as DATA_DESPACHO,
                                                    cast(delivered_date as date) as DATA_ENTREGA,
                                                    cast(os.sale_date as date) as DATA_PEDIDO,
                                                    cast(osh.billing_date as date) as DATA_FATURADO,
                                                    --cast(payment_confirmation_date as date) as DATA_CONFIR_PGTO,
                                                    cast(first_date_payment as date) as DATA_APROV,
                                                    cast(expected_delivery_date as date) as DATA_PROMETIDA,
                                                    store_code as UNI_NEG,
                                                    dead_line_in_days as DIAS_PRAZO,
                                                    osh.shipping_status_name as STATUS, 
                                                    shipping_address_city as MUNICIPIO,
                                                    shipping_address_state_code as ESTADO,
                                                    ad.postal_code as CEP,
                                                    n.Ns_Entregas as Tipo_Envio,
                                                    (case when d.default_handling_time is null then 0 else (d.default_handling_time/24) end) as dias_prazo_nse,
                                                    --sum((shp.value_placed) + (case when shp.seller_id != '0' then shp.value_freight else 0 end)) as VLR,
                                                    --sum(case when osh.billing_date is null then 0 else (shp.value_placed) + (case when shp.seller_id != '0' then osh.freight else 0 end) end) as VLR,
                                                    --sum(shp.value_freight) as FRETE,
                                                    shp.value_placed as VLR,
                                                    shp.value_freight as frete
                                                    --ck.Status_checkin
                                                FROM `maga-bigdata.netshoes_dw.vw_order_full` os, 
                                                unnest(os.order_shipping) osh,  unnest(osh.shipping_product) shp, unnest(shipping_status_history) hist
                                                left join maga-bigdata.netshoes_dw.vw_order_address_shipping ad on ad.order_id = os.order_id and ad.order_shipping_id = osh.order_shipping_id
                                                left join maga-bigdata.marketplace_analytics.sellers_nets_nse as n on n.ID_SELLER = shp.seller_id
                                                left join (SELECT
                                                                --id,
                                                                seller_name,
                                                                default_handling_time,
                                                                name
                                                            FROM
                                                                maga-bigdata.maga_onboarding.distribution_center dc
                                                            JOIN
                                                                maga-bigdata.maga_onboarding.handling_time ht ON ht.id = dc.handling_time_id
                                                            JOIN
                                                                maga-bigdata.maga_onboarding.origin_address oa ON oa.id = dc.address_id
                                                            JOIN
                                                                maga-bigdata.maga_onboarding.distribution_center_organizations dco ON dco.distributioncenter_id = dc.id
                                                            JOIN
                                                                maga-bigdata.maga_onboarding.organization o ON o.id = dco.organization_id
                                                                where name='netshoes'
                                                                group by seller_name,default_handling_time, name) as d on d.seller_name = shp.seller_id

                                                left join (SELECT DISTINCT
                                                                    A.PEDS_COD COD_PEDIDO,
                                                                    --A.PEDS_EXT_COD PEDIDO_EXTERNO,
                                                                    --B.TROD_COD CODIGO_DA_TROCA,
                                                                    B.TROD_STR_STA Status_checkin,
                                                                    --C.ITET_DAT_PROENT DATA_CHECKIN
                                                                    FROM `maga-bigdata.abacos_nets.aba_tcom_pedsai` A
                                                                    INNER JOIN `maga-bigdata.abacos_nets.aba_ttrd_trodev` B ON A.CLIF_COD = B.CLIF_COD
                                                                    --INNER JOIN `maga-bigdata.abacos_nets.aba_ttrd_itetrd` C ON B.TROD_COD = C.TROD_COD
                                                                    --WHERE A.PEDS_COD IN ('XXX')
                                                                    WHERE B.TROD_STR_STA in ('Finalizado', 'Aberto')	
                                                                        --AND PEDS_VAL_PED = ITET_VAL_PRETOT
                                                                        --ITET_DAT_PROENT IS NOT NULL
                                                                        --ITET_DAT_PROENT >='2019-01-01'
                                                                        --AND B.TROD_DAT_PRAPRO >= '2021-09-01'
                                                                        --and A.PEDS_COD = 864944870
                                                                    group by A.PEDS_COD, B.TROD_STR_STA) as ck on cast(ck.COD_PEDIDO as string) = osh.delivery_id             
                                                        where
                                                        cast(os.sale_date as date) >= '{}'
                                                        --EXTRACT(YEAR FROM date(expected_delivery_date)) = 2022
                                                        --and EXTRACT(MONTH FROM date(expected_delivery_date)) = 06
                                                        --and date(expected_delivery_date) <= current_date() 
                                                        and os.code not like '%T%'
                                                        and shp.source = '3P' 
                                                        and os.is_ignore_pickup = false
                                                        and NOT(os.amount >= 50000 AND osh.status = 12) 
                                                        and IFNULL(cancel_code,0) <> 100 
                                                        and osh.shipping_status_name in ('CANCELADO', 'DESPACHADO', 'ENTREGUE', 'FATURADO', 'PRONTO PARA FATURAR')
                                                        and ck.Status_checkin is null
                                                        group by 
                                                        1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20""".format(data_base))

###################### Retirada pedidos com checkin - Base original #########################################################################################
#pedidos_checkin = pd.merge(pedidos,pedidos_ckeckin, how='left', on='COD_PEDIDO' )
#base_pedidos = pedidos_checkin.query('STATUS_checkin!=["Finalizado","Aberto"]')
pedidos['STATUS_checkin'] = ''

base_pedidos = pedidos.copy()

# Formatar campos de data
base_pedidos['DATA_APROV'] = base_pedidos['DATA_APROV'].astype(object)
base_pedidos['DATA_APROV'] = pd.to_datetime(base_pedidos['DATA_APROV'])
base_pedidos['mes_data_aprovacao'] = base_pedidos['DATA_APROV'].dt.month
base_pedidos['DIAS_PRAZO'] = base_pedidos['DIAS_PRAZO'].replace('',0)
base_pedidos['DATA_PEDIDO'] = base_pedidos['DATA_PEDIDO'].astype(object)
base_pedidos['DATA_PEDIDO'] = pd.to_datetime(base_pedidos['DATA_PEDIDO'])
base_pedidos['MES_DATA_PEDIDO'] = base_pedidos['DATA_PEDIDO'].dt.month
base_pedidos['DATA_ENTREGA'] = pd.to_datetime(base_pedidos['DATA_ENTREGA'])
base_pedidos['DATA_DESPACHO'] = pd.to_datetime(base_pedidos['DATA_DESPACHO'])
base_pedidos['Hoje'] = datetime.datetime.today().strftime('%Y-%m-%d')
base_pedidos['Hoje'] = pd.to_datetime(base_pedidos['Hoje'], format='%Y-%m-%d')
#Formatar colunas Lojista e Id seller
base_pedidos['STATUS'] = base_pedidos['STATUS'].str.strip()
base_pedidos['LOJISTA'] = base_pedidos['LOJISTA'].str.lower()
base_pedidos['ID_SELLER'] = base_pedidos['ID_SELLER'].str.strip()
base_pedidos['ID_SELLER'] = base_pedidos['ID_SELLER'].astype(str)

base_pedidos['DATA_PROMETIDA'] = pd.to_datetime(base_pedidos['DATA_PROMETIDA'])
base_pedidos['mes_data_prometida'] = base_pedidos['DATA_PROMETIDA'].dt.month

# Sellers blocado
sellers_blocado = ['13514',
                  '7968',
                  '9321',
                  '3328',
                  '1483',
                  '4761',
                  '6749',
                  '13590',
                  '12806',
                  '4901',
                  '5382',
                  '2281',
                  '12114',
                  '3468',
                  '10641',
                  '4282',
                  '3441',
                  '5021',
                  '5941',
                  '581',
                  '9963',
                  '13268',
                  '4188',
                  '7930',
                  '12666',
                  '12739',
                  '4764',
                  '7861',
                  '12865',
                  '11932',
                  '7644',
                  '12089',
                  '3464',
                  '6721',
                  '13201',
                  '2082',
                  '3421',
                  '12814',
                  '12799',
                  '5601',
                  '7934',
                  '12855',
                  '11956',
                  '12130',
                  '9501',
                  '4230',
                  '12844',
                  '14699',
                  '581',
                  '14673',
                  '14630',
                  '13268',
                  '14278',
                  '14587',
                  '14887',
                  '14550',
                  '14970',
                  '15182',
                  '14897',
                  '15057',
                  '14967',
                  '1356',
                  '3131',
                  '15237',
                  '15173',
                  '13002',
                  '15244',
                  '15116',
                  '15574',
                  '15510',
                  '15269',
                  '14697',
                  '3385',
                  '15812',
                  '16164',
                  '15971',
                  '16243',
                  '15206',
                  '16273',
                  '16116',
                  '14468',
                  '16755',
                  '17115',
                  '14755',
                  '17008',
                  '16151',
                  '17274',
                  '17471']
#Função para coluna blocado
def seller_blocado(base_pedidos):
  if (base_pedidos['ID_SELLER'] in sellers_blocado):
    return 'sim'
  else:
    return 'não' 
# Coluna Blocado
base_pedidos['Blocado'] = base_pedidos.apply(seller_blocado, axis=1)

################################################################# Base pedidos acumulado ##########################################################

# Base Pedidos acumulada sem cancelamento
base_pedidos_mes = base_pedidos.query('STATUS!="CANCELADO"').drop_duplicates()

# Total pedidos
base_seller_lojista = base_pedidos_mes[['ID_SELLER','LOJISTA', 'Tipo_Envio']]
base_tt_pedidos = pd.DataFrame(pd.pivot_table(base_pedidos_mes, index=['ID_SELLER','LOJISTA', 'Tipo_Envio'], columns='STATUS', values='COD_PEDIDO', aggfunc='count',margins=True, margins_name='TT_Pedidos')).fillna(0)

################################################################## Pedidos Entregues ################################################################
# Pedidos Entregues
base_pedidos_entregues = base_pedidos_mes.query('STATUS=="ENTREGUE"')
base_pedidos_entregues['Media_Entregas_Dias'] = (base_pedidos_entregues['DATA_ENTREGA']-base_pedidos_entregues['DATA_PROMETIDA'])
base_pedidos_entregues['Media_Entregas_Dias'] = base_pedidos_entregues['Media_Entregas_Dias'].dt.days
def analise_pedidos(base_pedidos_entregues):
  if base_pedidos_entregues['DATA_ENTREGA'] <= base_pedidos_entregues['DATA_PROMETIDA']:
    return 'entregue_dentro_prazo'
  else:
    return 'entregue_fora_prazo'
base_pedidos_entregues['status_entrega'] = base_pedidos_entregues.apply(analise_pedidos, axis=1)
resumo_base_pedidos_entregues = pd.DataFrame(pd.pivot_table(base_pedidos_entregues, index=['ID_SELLER','LOJISTA'], columns='status_entrega', values='COD_PEDIDO', aggfunc='count', margins=True, margins_name='TT_Pedidos_Entregues').fillna(0))


# Prazo médio de entrega (Dias)
resumo_base_pedidos_dias = pd.DataFrame(pd.pivot_table(base_pedidos_entregues, index=['ID_SELLER','LOJISTA'], values='Media_Entregas_Dias', aggfunc='mean').fillna(0))
resumo_base_pedidos_dias['Media_Entregas_Dias'] = round(resumo_base_pedidos_dias['Media_Entregas_Dias'],2)


# Prazo médio prometido
resumo_base_pedidos_prazo_dias = pd.DataFrame(pd.pivot_table(base_pedidos_entregues, index=['ID_SELLER','LOJISTA'], values='DIAS_PRAZO', aggfunc='mean').fillna(0))
resumo_base_pedidos_prazo_dias['DIAS_PRAZO'] = round(resumo_base_pedidos_prazo_dias['DIAS_PRAZO'],2)

##################################################################### Pedidos Despachados ################################################################

# Base pedidos despachado
base_pedidos_despachado = base_pedidos_mes.query('STATUS==["DESPACHADO","ENTREGUE"] & DATA_DESPACHO!=""').drop_duplicates()
# Função limite de postagem
def data_limite_postagem(base_pedidos_despachado):
  if base_pedidos_despachado['Tipo_Envio'] == 'NSE':
    return wd.workdays(base_pedidos_despachado['DATA_APROV'], base_pedidos_despachado['dias_prazo_nse'], country='BR')
  elif base_pedidos_despachado['Tipo_Envio'] == 'Não' and base_pedidos_despachado['Blocado'] == 'não': 
    return wd.workdays(base_pedidos_despachado['DATA_APROV'], 2, country='BR')
  elif base_pedidos_despachado['Tipo_Envio'] == 'Não' and base_pedidos_despachado['Blocado'] == 'sim': 
    return wd.workdays(base_pedidos_despachado['DATA_APROV'], 5, country='BR')  
  else:
      return ""  
base_pedidos_despachado['DATA_LIMITE_POSTAGEM'] = base_pedidos_despachado.apply(data_limite_postagem, axis=1)

# Função análise postagem
def analise_postagem(base_pedidos_despachado):
  if base_pedidos_despachado['DATA_DESPACHO'] <= base_pedidos_despachado['DATA_LIMITE_POSTAGEM']:
    return 'postagem_dentro_prazo'
  else:
    return 'postagem_fora_prazo'
base_pedidos_despachado['status_postagem'] = base_pedidos_despachado.apply(analise_postagem, axis=1)
resumo_base_pedidos_despachado = pd.DataFrame(pd.pivot_table(base_pedidos_despachado, index=['ID_SELLER','LOJISTA'], columns='status_postagem', values='COD_PEDIDO', aggfunc='count', margins=True, margins_name='TT_Pedidos_Despachados').fillna(0))

base_pedidos_despachado_v2 = base_pedidos_despachado.query('STATUS=="DESPACHADO"')

# Gravar na tabela pedidos despachados
dados.grava_tabela_drive('Tb_pedidos_despachados_nets', base_pedidos_despachado_v2)


##################################################################### Pedidos Faturados e Pronto para Faturar ################################################################

# Base Pedidos Faturados
base_pedidos_faturado = base_pedidos_mes.query('STATUS=="FATURADO"')
# Função limite postagem
def data_limite_postagem(base_pedidos_faturado):
  if base_pedidos_faturado['Tipo_Envio'] == 'NSE':
    return wd.workdays(base_pedidos_faturado['DATA_APROV'], base_pedidos_faturado['dias_prazo_nse'], country='BR')
  elif base_pedidos_faturado['Tipo_Envio'] == 'Não' and base_pedidos_faturado['Blocado'] == 'não': 
    return wd.workdays(base_pedidos_faturado['DATA_APROV'], 2, country='BR')
  elif base_pedidos_faturado['Tipo_Envio'] == 'Não' and base_pedidos_faturado['Blocado'] == 'sim': 
    return wd.workdays(base_pedidos_faturado['DATA_APROV'], 5, country='BR')
  else:
      return ""    
base_pedidos_faturado['DATA_LIMITE_POSTAGEM'] = base_pedidos_faturado.apply(data_limite_postagem, axis=1)

# Base Pedidos Pronto para faturar

base_pedidos_pronto_faturar = base_pedidos_mes.query('STATUS=="PRONTO PARA FATURAR" & DATA_APROV!=""')
# Função limite postagem
def data_limite_postagem(base_pedidos_pronto_faturar):
  if base_pedidos_pronto_faturar['Tipo_Envio'] == 'NSE':
    return wd.workdays(base_pedidos_pronto_faturar['DATA_APROV'], base_pedidos_pronto_faturar['dias_prazo_nse'], country='BR')
  elif  base_pedidos_pronto_faturar['Tipo_Envio'] == 'Não' and base_pedidos_pronto_faturar['Blocado'] == 'não': 
    return wd.workdays(base_pedidos_pronto_faturar['DATA_APROV'], 2, country='BR')
  elif  base_pedidos_pronto_faturar['Tipo_Envio'] == 'Não' and base_pedidos_pronto_faturar['Blocado'] == 'sim': 
    return wd.workdays(base_pedidos_pronto_faturar['DATA_APROV'], 5, country='BR')  
  else:
      return ""  
base_pedidos_pronto_faturar['DATA_LIMITE_POSTAGEM'] = base_pedidos_pronto_faturar.apply(data_limite_postagem, axis=1)

# Concatenação bases
pedidos_atraso = pd.concat([base_pedidos_faturado, base_pedidos_pronto_faturar]).drop_duplicates()

######################################################################## Pedidos Cancelados #################################################################################

# Base pedidos Cancelados
base_pedidos['validacao_data'] = base_pedidos['DATA_APROV'].fillna('sem_data')

def excluir_pedidos_cancelados(base_pedidos):
    if base_pedidos['STATUS'] == 'CANCELADO' and base_pedidos['validacao_data']=='sem_data':
        return 'excluir'
    else:
        return 'ok'

base_pedidos['validacao_pedidos'] = base_pedidos.apply(excluir_pedidos_cancelados, axis=1)

base_pedidos_v2 = base_pedidos.query('validacao_pedidos!="excluir"')
base_pedidos_v2.drop(columns=["validacao_pedidos", "validacao_data"], inplace=True)

base_cancelados = base_pedidos_v2.query('STATUS=="CANCELADO"')
pedidos_cancelados = pd.DataFrame(pd.pivot_table(base_cancelados, index=['ID_SELLER','LOJISTA'], values='COD_PEDIDO_LOJA', aggfunc='count').fillna(0))
pedidos_cancelados.rename(columns={'COD_PEDIDO_LOJA':'Pedidos_Cancelados'}, inplace=True)

######################################################################### Pedidos Aprovados #################################################################################
# Base Pedidos Aprovados
base_aprovados = base_pedidos.query('STATUS!="CANCELADO"').drop_duplicates(subset=['COD_PEDIDO'])
pedidos_aprovados = pd.DataFrame(pd.pivot_table(base_aprovados, index=['ID_SELLER','LOJISTA'], values='DATA_APROV', aggfunc='count').fillna(0))
pedidos_aprovados.rename(columns={'DATA_APROV':'Pedidos_Aprovados'}, inplace=True)


######################################################################### Dados KA Nets ######################################################################################
dados_ka = dados.consulta_bigquery('maga-bigdata', """select * from maga-bigdata.nets_gestao_info.tgi_seller_group""")
dados_ka.rename(columns={'group_owner':'Assistido', 'code':'ID_SELLER'}, inplace=True)
dados_ka['Assistido'] = dados_ka['Assistido'].str.replace('Não Assistido', 'Não_Assistido')
# Função sellers ka
def sellers_ka(dados_ka):
  if dados_ka['Assistido'] == 'Assistido':
    return "Key Account"
  else:
    return 'Long Tail'
dados_ka['Key Account?']  = dados_ka.apply(sellers_ka, axis=True)
dados_ka_final = dados_ka[['ID_SELLER', 'Key Account?']]
dados_ka_final['ID_SELLER'] = dados_ka_final['ID_SELLER'].apply(str)
dados_ka_final['ID_SELLER'] = dados_ka_final['ID_SELLER'].str.strip()


######################################################################### Base Protocolos #################################################################################
df_protocolos = dados.busca_tabela_drive('Tabela_protocolos_sac', 'Página1')

base_protocolos = df_protocolos[['Atendimento Codigo', 
                                  'Data Inicial', 
                                  'Data Final', 
                                  'Servico', 
                                  'Estagio Atual', 
                                  'Lojista Ajustado']]

base_protocolos.rename(columns={'Atendimento Codigo':'Atendimento_Codigo',
                                'Data Inicial':'Data_Inicial',
                                'Data Final':'Data_Final',
                                'Estagio Atual':'Estagio_Atual',
                                'Lojista Ajustado':'Lojista_Ajustado'}, inplace=True)                                  

base_protocolos['LOJISTA'] = base_protocolos['Lojista_Ajustado'].str.lower().str.strip()
base_protocolos.rename(columns={'Atendimento_Codigo':'Protocolo'}, inplace=True)
dinamica_protocolo = pd.pivot_table(base_protocolos, index='LOJISTA', values='Protocolo', aggfunc='count', margins=True)

# Grava tabela
#grava_tabela_drive('Tb_protocolos_nets', base_protocolos)
base_protocolos.to_gbq(destination_table='data.Tb_protocolos_nets',project_id=projeto_gcp_2, if_exists='replace')

######################################################################### Dados CR e Nps #################################################################################

dados_cr_nps = dados.consulta_bigquery(projeto_gcp_1, """SELECT Seller_id,
                                                    round((SAFE_DIVIDE((SUM(abac_App_Site)+SUM(abac_B2B)+SUM(abac_Cadastro)+SUM(abac_Campanhas)+SUM(abac_Entrega)+SUM(abac_Ncard)+SUM(abac_Pedido)+SUM(abac_Produto)+SUM(abac_TVC)+SUM(abac_Outros)),SUM(Captado))),3) as CR_Seller,
                                                    round((SAFE_DIVIDE((SUM(nps_promotor) - SUM(nps_detrator)),(sum(nps_detrator)+SUM(nps_promotor)+SUM(nps_neutro)))*100),2) as NPS_Seller
                                                    FROM maga-bigdata.sac_inteligencia_clientes.sellers_netshoes_indicadores
                                                        WHERE seller_id is not null
                                                        AND data >= '{}'
                                                        GROUP BY Seller_id""".format(data_base))

#dados_cr_nps['CR_Seller'] = round((dados_cr_nps['CR_Seller']*100),2)
dados_cr_nps['CR_Seller'] = pd.to_numeric(dados_cr_nps['CR_Seller']).fillna(0)
dados_cr_nps['NPS_Seller'] = pd.to_numeric(dados_cr_nps['NPS_Seller']).fillna(0)
dados_cr_nps['LOJISTA'] = dados_cr_nps['Seller_id'].str.lower()
dinamica_cr_nps = dados_cr_nps[['LOJISTA', 'CR_Seller', 'NPS_Seller']].drop_duplicates(subset='LOJISTA')

######################################################################### Base Hubsales #################################################################################
# Import base
#'/content/drive/MyDrive/Ciclo Pedidos Nets Novo/sellers Hubsales - BF.xlsx'
base_husales = pd.read_excel('/content/drive/MyDrive/Colab_Notebooks/arquivos/ciclo_pedido/nets/sellers Hubsales - BF.xlsx', usecols=['Id Seller', 
                                                                                                                           'Nome Loja', 
                                                                                                                           'Hubsales?']).fillna(0)
# Ajustes colunas
base_husales.rename(columns={'Id Seller':'ID_SELLER', 'Nome Loja':'LOJISTA'}, inplace=True)
base_husales['ID_SELLER'] = base_husales['ID_SELLER'].apply(int).apply(str)
base_husales['LOJISTA'] = base_husales['LOJISTA'].str.lower()
base_husales_final = base_husales[['ID_SELLER', 'Hubsales?']]


######################################################################### Concatenação das bases #################################################################################

concatencao_1 = pd.merge(base_seller_lojista,base_tt_pedidos, how='left', on=['ID_SELLER','LOJISTA','Tipo_Envio']).fillna(0).drop_duplicates()
concatencao_2 = pd.merge(concatencao_1,resumo_base_pedidos_entregues, how='left', on=['ID_SELLER','LOJISTA']).fillna(0).drop_duplicates()
concatencao_3 = pd.merge(concatencao_2,resumo_base_pedidos_despachado, how='left', on=['ID_SELLER','LOJISTA']).fillna(0).drop_duplicates()
concatencao_4 = pd.merge(concatencao_3,pedidos_cancelados, how='left', on=['ID_SELLER']).fillna(0).drop_duplicates()
concatencao_5 = pd.merge(concatencao_4,pedidos_aprovados, how='left', on=['ID_SELLER','LOJISTA']).fillna(0).drop_duplicates()
concatencao_6 = pd.merge(concatencao_5,dados_ka_final, how='left', on=['ID_SELLER']).fillna('Não').drop_duplicates()
#concatencao_7 = pd.merge(concatencao_6,base_nse_final, how='left', on=['ID_SELLER']).fillna('Não').drop_duplicates()
concatencao_7 = pd.merge(concatencao_6,dinamica_protocolo, how='left', on=['LOJISTA']).fillna(0).drop_duplicates()
concatencao_8 = pd.merge(concatencao_7,dinamica_cr_nps, how='left', on=['LOJISTA']).fillna(0).drop_duplicates()
concatencao_9 = pd.merge(concatencao_8,base_husales_final, how='left', on=['ID_SELLER']).fillna("Não").drop_duplicates()
concatencao_10 = pd.merge(concatencao_9,resumo_base_pedidos_dias, how='left', on=['ID_SELLER','LOJISTA']).fillna(0).drop_duplicates()
concatencao_11 = pd.merge(concatencao_10,resumo_base_pedidos_prazo_dias, how='left', on=['ID_SELLER','LOJISTA']).fillna(0).drop_duplicates()
concatencao_12 = pd.merge(concatencao_11, reclamacoes_formais_resumo, how='left', on='LOJISTA').fillna(0)

 ######################################################################### Indicador de reputação #################################################################################
# Criar colunas de participação
concatencao_12['% Entrega'] = round((concatencao_12['entregue_dentro_prazo']/concatencao_12['TT_Pedidos'])*100,2)
concatencao_12['% Postagem'] = round((concatencao_12['postagem_dentro_prazo']/concatencao_12['TT_Pedidos'])*100,2)
concatencao_12['% Cancelados'] = round((concatencao_12['Pedidos_Cancelados']/concatencao_12['TT_Pedidos'])*100,2)
concatencao_12['% Reclamação'] = round((concatencao_12['Qtd_Reclamacoes_Formais']/concatencao_12['TT_Pedidos'])*100,2)

# Classificação  regua entrega, despacho, cancelados, reclamação e nps
def regra_entrega(concatencao_12):
  if concatencao_12['% Entrega'] >= 0.0 and concatencao_12['% Entrega'] <= 84.99:
    return 1
  elif concatencao_12['% Entrega'] >= 85 and concatencao_12['% Entrega'] <= 88.99:
    return 2
  elif concatencao_12['% Entrega'] >= 89 and concatencao_12['% Entrega'] <= 92.99:
    return 3
  elif concatencao_12['% Entrega'] >= 93 and concatencao_12['% Entrega'] <= 94.99:
    return 4
  elif concatencao_12['% Entrega'] >= 95:
    return 5
concatencao_12['reg_entrega'] = concatencao_12.apply(regra_entrega, axis=1)


def regra_despacho(concatencao_12):
  if concatencao_12['% Postagem'] >= 0.0 and concatencao_12['% Postagem'] <= 84.99:
    return 1
  elif concatencao_12['% Postagem'] >= 85 and concatencao_12['% Postagem'] <= 88.99:
    return 2
  elif concatencao_12['% Postagem'] >= 89 and concatencao_12['% Postagem'] <= 92.99:
    return 3
  elif concatencao_12['% Postagem'] >= 93 and concatencao_12['% Postagem'] <= 94.99:
    return 4
  elif concatencao_12['% Postagem'] >= 95:
    return 5
concatencao_12['reg_Postagem'] = concatencao_12.apply(regra_despacho, axis=1)


def regra_cancelados(concatencao_12):
  if concatencao_12['% Cancelados'] <= 2.0:
    return 5
  elif concatencao_12['% Cancelados'] >= 2.01 and concatencao_12['% Cancelados'] <= 3.0:
    return 4
  elif concatencao_12['% Cancelados'] >= 3.01 and concatencao_12['% Cancelados'] <= 4.0:
    return 3
  elif concatencao_12['% Cancelados'] >= 4.01 and concatencao_12['% Cancelados'] <= 5.0:
    return 2
  elif concatencao_12['% Cancelados'] >= 5.01:
    return 1
concatencao_12['reg_Cancelados'] = concatencao_12.apply(regra_cancelados, axis=1)


def regra_reclamacao(concatencao_12):
  if concatencao_12['% Reclamação'] <= 1.0:
    return 5
  elif concatencao_12['% Reclamação'] >= 1.01 and concatencao_12['% Reclamação'] <= 2.50:
    return 4
  elif concatencao_12['% Reclamação'] >= 2.51 and concatencao_12['% Reclamação'] <= 3.50:
    return 3
  elif concatencao_12['% Reclamação'] >= 3.51 and concatencao_12['% Reclamação'] <= 4.00:
    return 2
  elif concatencao_12['% Reclamação'] >= 4.01:
    return 1
concatencao_12['reg_Reclamação'] = concatencao_12.apply(regra_cancelados, axis=1)


def regra_nps(concatencao_12):
  if concatencao_12['NPS_Seller'] <= 64.9 :
    return 1
  elif concatencao_12['NPS_Seller'] >= 65.00 and concatencao_12['NPS_Seller'] <= 67.99:
    return 2
  elif concatencao_12['NPS_Seller'] >= 68.00 and concatencao_12['NPS_Seller'] <= 70.00:
    return 3
  elif concatencao_12['NPS_Seller'] >= 70.01 and concatencao_12['NPS_Seller'] <= 74.99:
    return 4
  elif concatencao_12['NPS_Seller'] >= 75.00:
    return 5
concatencao_12['reg_nps'] = concatencao_12.apply(regra_nps, axis=1)


def regra_cr(concatencao_12):
  if concatencao_12['CR_Seller'] >= 0.301 :
    return 1
  elif concatencao_12['CR_Seller'] >= 0.201 and concatencao_12['CR_Seller'] <= 0.300:
    return 2
  elif concatencao_12['CR_Seller'] >= 0.151 and concatencao_12['CR_Seller'] <= 0.200:
    return 3
  elif concatencao_12['CR_Seller'] >= 0.101 and concatencao_12['CR_Seller'] <= 0.150:
    return 4
  elif concatencao_12['CR_Seller'] <= 0.100:
    return 5
concatencao_12['reg_cr'] = concatencao_12.apply(regra_cr, axis=1)


# Indicador da reputação
def funcao_reputacao(concatencao_12):
  if concatencao_12['Tipo_Envio'] == 'Não':
    return round((concatencao_12['reg_nps']/5*1.45)+(concatencao_12['reg_entrega']/5*1.50)+(concatencao_12['reg_cr']/5*0.50)+(concatencao_12['reg_Reclamação']/5*0.35)+(concatencao_12['reg_Cancelados']/5*1.20),2)
  elif concatencao_12['Tipo_Envio'] == 'NSE':
    return round((concatencao_12['reg_nps']/5*1.45)+(concatencao_12['reg_Postagem']/5*1.50)+(concatencao_12['reg_cr']/5*0.50)+(concatencao_12['reg_Reclamação']/5*0.35)+(concatencao_12['reg_Cancelados']/5*1.20),2)
concatencao_12['Indicador_Reputacao'] = concatencao_12.apply(funcao_reputacao, axis=1)

######################################################################### Base pedidos atrasados #################################################################################
# Base Pedidos em atrasados
reputacao = concatencao_12[['ID_SELLER', 'Indicador_Reputacao']]
pedidos_atraso_reputacao = pd.merge(pedidos_atraso, reputacao, how='left', on='ID_SELLER')
# Coluna ontem
pedidos_atraso_reputacao['Ontem'] = pedidos_atraso_reputacao['Hoje'] - datetime.timedelta(days = 1)
# Coluna Pedidos Atrasados
def pedidos_em_atraso(pedidos_atraso_reputacao):
  if pedidos_atraso_reputacao['DATA_PROMETIDA'] <= pedidos_atraso_reputacao['Ontem']: 
    return 'Pedido_Atrasado'
  else:
    return ''
pedidos_atraso_reputacao['Pedidos_Atrasados'] =  pedidos_atraso_reputacao.apply(pedidos_em_atraso, axis=1)   

# Coluna Expedição  Atrasada
def expedicao_em_atraso(pedidos_atraso_reputacao):
  if pedidos_atraso_reputacao['DATA_LIMITE_POSTAGEM'] <= pedidos_atraso_reputacao['Ontem']: 
    return 'Expedicao_Atrasada'
  else:
    return ''
pedidos_atraso_reputacao['Pedidos_Expedicao_Atrasada'] =  pedidos_atraso_reputacao.apply(expedicao_em_atraso, axis=1)

# Grava tabela 
dados.grava_tabela_drive('Tb_pedidos_atrasados',pedidos_atraso_reputacao)

######################################################################### Resumo pedidos atrasados e expedição atrasados #################################################################################

base1 = pedidos_atraso_reputacao.query('Pedidos_Atrasados=="Pedido_Atrasado"').drop_duplicates(subset=['ID_SELLER', 'COD_PEDIDO_LOJA'])
resumo_base1 = pd.pivot_table(base1, index='ID_SELLER', values=['Pedidos_Atrasados'], aggfunc='count')
base2 = pedidos_atraso_reputacao.query('Pedidos_Expedicao_Atrasada=="Expedicao_Atrasada"').drop_duplicates(subset=['ID_SELLER', 'COD_PEDIDO_LOJA'])
resumo_base2 = pd.pivot_table(base2, index='ID_SELLER', values=['Pedidos_Expedicao_Atrasada'], aggfunc='count')

######################################################################### Concatenação das bases v2 #################################################################################
 
concatencao_14 = pd.merge(concatencao_12,resumo_base1, how='left', on='ID_SELLER')
concatencao_15 = pd.merge(concatencao_14,resumo_base2, how='left', on='ID_SELLER')

######################################################################### Base GMV ###################################################################################################

base_pedidos_mes_gmv = pd.pivot_table(base_pedidos_mes, index='ID_SELLER', values='VLR', aggfunc='sum')
base_pedidos_mes_gmv.rename(columns={'VLR':'GMV'}, inplace=True)

######################################################################### Concatenação das bases v3 #################################################################################

concatencao_16 = pd.merge(concatencao_15, base_pedidos_mes_gmv, how='left', on='ID_SELLER')
concatencao_16['GMV'] = concatencao_16['GMV'].astype(float)
concatencao_16['GMV'] = round(concatencao_16['GMV'],2)
# GRavar tabela Resumo
dados.grava_tabela_drive('Tb_Pedidos_Nets_Novo', concatencao_16)

######################################################################### Resumo mensal ciclo pedido #################################################################################

# Base Pedidos resumo ciclo - mês a mês
base_pedidos_ciclo = base_pedidos.query('STATUS!="CANCELADO" & MES_DATA_PEDIDO==6').drop_duplicates()
# Total pedidos
tt_pedidos = pd.pivot_table(base_pedidos_ciclo, index=['ID_SELLER', 'LOJISTA'], values='COD_PEDIDO', aggfunc='count').fillna(0)
tt_pedidos.rename(columns={'COD_PEDIDO':'TT_pedidos'}, inplace=True)
# Pedidos aprovados
pedidos_aprovados_v2 = pd.DataFrame(pd.pivot_table(base_pedidos_ciclo, index=['ID_SELLER','LOJISTA'], values='DATA_APROV', aggfunc='count').fillna(0))
pedidos_aprovados_v2.rename(columns={'DATA_APROV':'Pedidos_Aprovados'}, inplace=True)
# Pedidos faturado
base_pedidos_ciclo_faturado = base_pedidos.query('MES_DATA_PEDIDO==6 & STATUS=="FATURADO"').drop_duplicates()
base_pedidos_ciclo_faturado_v2 = pd.pivot_table(base_pedidos_ciclo_faturado, index=['ID_SELLER', 'LOJISTA'], values='COD_PEDIDO', aggfunc='count').fillna(0)
base_pedidos_ciclo_faturado_v2.rename(columns={'COD_PEDIDO':'Pedidos_Faturados'}, inplace=True)
# Pedidos pronto para faturar
base_pedidos_ciclo_pronto = base_pedidos.query(' MES_DATA_PEDIDO==6 & STATUS=="PRONTO PARA FATURAR"').drop_duplicates()
base_pedidos_ciclo_pronto_v2 = pd.pivot_table(base_pedidos_ciclo_pronto, index=['ID_SELLER', 'LOJISTA'], values='COD_PEDIDO', aggfunc='count').fillna(0)
base_pedidos_ciclo_pronto_v2.rename(columns={'COD_PEDIDO':'Pedidos_pronto_faturar'}, inplace=True)
# Pedidos Expedição atrasada
pedidos_expedicao = pedidos_atraso_reputacao.query('MES_DATA_PEDIDO==6 & Pedidos_Expedicao_Atrasada=="Expedicao_Atrasada"').drop_duplicates()
pedidos_expedicao_v2 = pd.pivot_table(pedidos_expedicao, index='ID_SELLER', values=['Pedidos_Expedicao_Atrasada'], aggfunc='count')

# Concatenação das bases
concat_ciclo = pd.merge(base_seller_lojista, tt_pedidos, how='left', on=['ID_SELLER', 'LOJISTA']).fillna(0).drop_duplicates()
concat_ciclo_v2 = pd.merge(concat_ciclo, pedidos_aprovados_v2, how='left', on=['ID_SELLER', 'LOJISTA']).fillna(0).drop_duplicates()
concat_ciclo_v3 = pd.merge(concat_ciclo_v2, base_pedidos_ciclo_faturado_v2, how='left', on=['ID_SELLER', 'LOJISTA']).fillna(0).drop_duplicates()
concat_ciclo_v4 = pd.merge(concat_ciclo_v3, base_pedidos_ciclo_pronto_v2, how='left', on=['ID_SELLER', 'LOJISTA']).fillna(0).drop_duplicates()
concat_ciclo_v5 = pd.merge(concat_ciclo_v4, pedidos_expedicao_v2, how='left', on=['ID_SELLER']).fillna(0).drop_duplicates()

# Base protocolos mês
base_protocolos_ciclo_v2 = dados.consulta_bigquery('marketplace-analytics-333712', """SELECT * FROM `marketplace-analytics-333712.data.Tb_protocolos_nets` 
                                                                                WHERE parse_date('%d/%m/%Y', Data_Inicial) >= '2022-06-01'
                                                                                and Data_Inicial not in ('AtendimentoDataInicial')""")
dinamica_protocolo_v2 = pd.pivot_table(base_protocolos_ciclo_v2, index='LOJISTA', values='Protocolo', aggfunc='count', margins=True)
concat_ciclo_v6 = pd.merge(concat_ciclo_v5, dinamica_protocolo_v2, how='left', on=['LOJISTA']).fillna(0).drop_duplicates()

# Pedidos enviados
base_pedidos_enviados = base_pedidos.query('STATUS==["ENTREGUE","DESPACHADO"]  & MES_DATA_PEDIDO==6').drop_duplicates()
base_pedidos_enviados_v2 = pd.pivot_table(base_pedidos_enviados, index=['ID_SELLER', 'LOJISTA'], values='COD_PEDIDO', aggfunc='count').fillna(0)
base_pedidos_enviados_v2.rename(columns={'COD_PEDIDO':'Pedidos_Enviados'}, inplace=True)
concat_ciclo_v7 = pd.merge(concat_ciclo_v6, base_pedidos_enviados_v2, how='left', on=['ID_SELLER', 'LOJISTA']).fillna(0).drop_duplicates()

# Gravar tabela
dados.grava_tabela_drive('Resumo_mes_ciclo_pedido', concat_ciclo_v7)
print('Ciclo pedido atualizado!')

Bloco executado


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:373: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:374: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:380: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

Ciclo pedido atualizado!


Análise Pedidos : Entregues e Despachados

In [ ]:
# Exportar para tabela BigQuery (análise pedido entregues)
#data_pedidos_entregues = (datetime.datetime.now() - datetime.timedelta(90)).strftime('%Y-%m-%d')
base_efetividade_entrega = dados.consulta_bigquery(projeto_gcp_1, """Select distinct
                                                                        shp.seller_name as LOJISTA,
                                                                        shp.seller_id as ID_SELLER,
                                                                        os.code as COD_PEDIDO_LOJA,
                                                                        osh.delivery_id as COD_PEDIDO,
                                                                        cast(dispatch_date as date) as DATA_DESPACHO,
                                                                        cast(delivered_date as date) as DATA_ENTREGA,
                                                                        cast(os.sale_date as date) as DATA_PEDIDO,
                                                                        cast(osh.billing_date as date) as DATA_FATURADO,
                                                                        --cast(payment_confirmation_date as date) as DATA_CONFIR_PGTO,
                                                                        cast(first_date_payment as date) as DATA_APROV,
                                                                        cast(expected_delivery_date as date) as DATA_PROMETIDA,
                                                                        store_code as UNI_NEG,
                                                                        dead_line_in_days as DIAS_PRAZO,
                                                                        osh.shipping_status_name as STATUS, 
                                                                        shipping_address_city as MUNICIPIO,
                                                                        shipping_address_state_code as ESTADO,
                                                                        ad.postal_code as CEP,
                                                                        n.Ns_Entregas as Tipo_Envio,
                                                                        --sum((shp.value_placed) + (case when shp.seller_id != '0' then shp.value_freight else 0 end)) as VLR,
                                                                        sum(case when osh.billing_date is null then 0 else (shp.value_placed) + (case when shp.seller_id != '0' then osh.freight else 0 end) end) as VLR,
                                                                        sum(shp.value_freight) as FRETE
                                                                    FROM `maga-bigdata.netshoes_dw.vw_order_full` os, 
                                                                    unnest(os.order_shipping) osh,  unnest(osh.shipping_product) shp, unnest(shipping_status_history) hist
                                                                    left join maga-bigdata.netshoes_dw.vw_order_address_shipping ad on ad.order_id = os.order_id and ad.order_shipping_id = osh.order_shipping_id
                                                                    left join maga-bigdata.marketplace_analytics.sellers_nets_nse as n on n.ID_SELLER = shp.seller_id
                                                                    
                                                                            where
                                                                            cast(os.sale_date as date) between '2022-04-01' and current_date()
                                                                            --EXTRACT(YEAR FROM date(expected_delivery_date)) = 2022
                                                                            --and EXTRACT(MONTH FROM date(expected_delivery_date)) = 06
                                                                            --and date(expected_delivery_date) <= current_date() 
                                                                            and os.code not like '%T%'
                                                                            and shp.source = '3P' 
                                                                            and os.is_ignore_pickup = false
                                                                            and NOT(os.amount >= 50000 AND osh.status = 12) 
                                                                            and IFNULL(cancel_code,0) <> 100 
                                                                            and osh.shipping_status_name in ('DESPACHADO', 'ENTREGUE', 'FATURADO', 'PRONTO PARA FATURAR')
                                                                            group by 
                                                                            1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17""")

base_efetividade_entrega.to_gbq(destination_table='marketplace_analytics.Tb_pedidos_entregues_nets', project_id=projeto_gcp_1, if_exists='replace')

print("\nTabela Entregues e Despachados - Atualizada!")

Nps Nets - Seller

In [ ]:
# Quantidade de pedidos por seller e por mês
quantidade_sellers = pd.pivot_table(base_pedidos_mes, index='LOJISTA', columns='mes_data_aprovacao', values='COD_PEDIDO_LOJA',aggfunc='count').fillna(0)
quantidade_sellers_concatencao_12 = pd.merge(concatencao_12, quantidade_sellers, how='left', on='LOJISTA')


dados.grava_tabela_drive('NPS_Seller', quantidade_sellers_concatencao_12)

print("Tabela NPS Nets - Atualizada!")

Acompanhamento Black

In [ ]:
"""
# Acompanhamento Black
#/content/drive/MyDrive/Ciclo Pedidos Nets Novo/Dash - Ciclo Pedidos Base.xlsx
dash = pd.read_excel('/content/drive/MyDrive/Ciclo Pedidos Nets Novo/Dash - Ciclo Pedidos Base.xlsx', sheet_name='Dash', skiprows=2,usecols=['Loja',
                                                                                                                                                'Ns Entregas?',
                                                                                                                                                'Hubsales',
                                                                                                                                                'Carteira',
                                                                                                                                                'Blocado?',
                                                                                                                                                'Status Seller',
                                                                                                                                                'Pausada?'])


# Renomear coluna 
dash.rename(columns={'Loja':'LOJISTA'}, inplace=True)

# Base de pedidos sem checkin
base_pedidos['STATUS'] = base_pedidos['STATUS'].str.strip() 
# Remover cancelados                                         
base_nets_original_sem_cancelados =  base_pedidos.query('STATUS!="CANCELADO" & STATUS!="ENTREGUE"')                                         
                          
base_nets_original_sem_cancelados['check_pedidos_loja'] = base_nets_original_sem_cancelados['COD_PEDIDO_LOJA'].str.contains("T")
base_1 = base_nets_original_sem_cancelados.query('check_pedidos_loja==False')


base_1['DIAS_PRAZO'] = base_1['DIAS_PRAZO'].replace('','0')
base_1['DIAS_PRAZO'] = base_1['DIAS_PRAZO'].astype('int64')
base_1['DATA_PEDIDO'] = base_1['DATA_PEDIDO'].astype(object)
base_1['DATA_PEDIDO'] = pd.to_datetime(base_1['DATA_PEDIDO'], format='%Y%m%d')
base_1['DATA_APROV'] = base_1['DATA_APROV'].astype(object)
base_1['DATA_APROV'] = pd.to_datetime(base_1['DATA_APROV'], format='%Y%m%d')

base_1['DATA_ENTREGA'] = base_1['DATA_ENTREGA'].astype(object)
base_1['DATA_ENTREGA'] = pd.to_datetime(base_1['DATA_ENTREGA'])

def data_dias_uteis(base_1):
    return wd.workdays(base_1['DATA_APROV'], base_1['DIAS_PRAZO'], country=None)

base_1['DATA_PROMETIDA'] = base_1.apply(data_dias_uteis, axis=1)
base_1['mes'] = base_1['DATA_PROMETIDA'].dt.month

base_1['hoje'] = datetime.datetime.today().strftime('%Y-%m-%d')
base_1['hoje'] = pd.to_datetime(base_1['hoje'])

base_2 = base_1.query('mes == 1')

# Tratamento dos dados

base_nets_s_entregues_dash = pd.merge(base_2, dash, how='left', on='LOJISTA')
base_nets_s_entregues_dash['DATA_APROV'] = pd.to_datetime(base_nets_s_entregues_dash['DATA_APROV'])
base_nets_s_entregues_dash['DAT_CRIA_PEDIDO'] = pd.to_datetime(base_nets_s_entregues_dash['DAT_CRIA_PEDIDO'], infer_datetime_format=True)

base_nets_s_entregues_dash.rename(columns={'Ns Entregas?':'Ns_Entregas',
                                           'Blocado?':'Blocado',
                                           'Status Seller':'Status_Seller',
                                           'Pausada?':'Pausada'}, inplace=True)

def data_limite_postagem(base_nets_s_entregues_dash):
  if base_nets_s_entregues_dash['Blocado'] == 'Blocado':
    return wd.workdays(base_nets_s_entregues_dash['DATA_APROV'], 5 , country=None)
  else:
    return wd.workdays(base_nets_s_entregues_dash['DATA_APROV'], 2 , country=None)

  
base_nets_s_entregues_dash['DATA_LIMITE_POSTAGEM'] = base_nets_s_entregues_dash.apply(data_limite_postagem, axis=1)

def nome(base_nets_s_entregues_dash):
  string_nova = unidecode.unidecode(base_nets_s_entregues_dash)
  return string_nova

base_nets_s_entregues_dash['LOJISTA'] = base_nets_s_entregues_dash['LOJISTA'].apply(nome)
base_nets_s_entregues_dash['LOJISTA'] = base_nets_s_entregues_dash['LOJISTA'].str.lower().str.replace(' ', '').str.replace("'","").str.replace('.','')

# Tratativa Municipio
base_nets_s_entregues_dash_v2 = base_nets_s_entregues_dash.query('MUNICIPIO!="566.914.590-91" & MUNICIPIO!="29780000" & MUNICIPIO!="28400-000" & MUNICIPIO!="89211-460"')
base_nets_s_entregues_dash_v2.head(1)


print("Gravando na tabela.........")

# Gravar na tabela dia atual
def input_drive():
    #Abrir planilha e primeira aba
    pagina = gc.open('Tb_acompanhamento_black_nets').sheet1
    #Limpar base
    pagina.clear()
    #Importar dataframe na planilha
    return set_with_dataframe(pagina, base_nets_s_entregues_dash_v2, include_column_header=True)
input_drive()
print("-----------------")
print("Tabela Atualizada")
print("-----------------")
"""

In [ ]:
'''
# Dados pedidos processados 3p 2021
# projeto
maga_bigdata = 'maga-bigdata'
#autenticação e select
client = bigquery.Client(project=maga_bigdata)
sql = """SELECT * FROM `maga-bigdata.marketplace_analytics.pedidos_processados_2021_3p`"""

# Select salvo na variavel
pedidos_2021 = client.query(sql).to_dataframe().fillna("")
pedidos_2021.head()

pedidos_2021['data_pedido_processado'] = pd.to_datetime(pedidos_2021['data_pedido_processado'])
pedidos_2021['mes'] = pedidos_2021['data_pedido_processado'].dt.month
resumo = pd.pivot_table(pedidos_2021, index='seller_id', columns='mes', values='pedido_processado', aggfunc='sum').fillna(0)
resumo.to_excel('/content/drive/MyDrive/dados_pedidos/Magalu/pedidos_processados_3p_2021.xlsx')

'''

In [ ]:
'''base_analise = dados.consulta_bigquery(projeto_gcp_1,"""Select distinct
                                                        shp.seller_name as LOJISTA,
                                                        shp.seller_id as ID_SELLER,
                                                        os.code as COD_PEDIDO_LOJA,
                                                        osh.delivery_id as COD_PEDIDO,
                                                        cast(dispatch_date as date) as DATA_DESPACHO,
                                                        cast(delivered_date as date) as DATA_ENTREGA,
                                                        cast(os.sale_date as date) as DATA_PEDIDO,
                                                        cast(osh.billing_date as date) as DATA_FATURADO,
                                                        --cast(payment_confirmation_date as date) as DATA_CONFIR_PGTO,
                                                        cast(first_date_payment as date) as DATA_APROV,
                                                        cast(expected_delivery_date as date) as DATA_PROMETIDA,
                                                        store_code as UNI_NEG,
                                                        dead_line_in_days as DIAS_PRAZO,
                                                        osh.shipping_status_name as STATUS, 
                                                        shipping_address_city as MUNICIPIO,
                                                        shipping_address_state_code as ESTADO,
                                                        ad.postal_code as CEP,
                                                        n.Ns_Entregas as Tipo_Envio,
                                                        (case when d.default_handling_time is null then 0 else (d.default_handling_time/24) end) as dias_prazo_nse,
                                                        --sum((shp.value_placed) + (case when shp.seller_id != '0' then shp.value_freight else 0 end)) as VLR,
                                                        sum(case when osh.billing_date is null then 0 else (shp.value_placed) + (case when shp.seller_id != '0' then osh.freight else 0 end) end) as VLR,
                                                        sum(shp.value_freight) as FRETE
                                                    FROM `maga-bigdata.netshoes_dw.vw_order_full` os, 
                                                    unnest(os.order_shipping) osh,  unnest(osh.shipping_product) shp, unnest(shipping_status_history) hist
                                                    left join maga-bigdata.netshoes_dw.vw_order_address_shipping ad on ad.order_id = os.order_id and ad.order_shipping_id = osh.order_shipping_id
                                                    left join maga-bigdata.marketplace_analytics.sellers_nets_nse as n on n.ID_SELLER = shp.seller_id
                                                    left join (SELECT
                                                                    --id,
                                                                    seller_name,
                                                                    default_handling_time,
                                                                    name
                                                                FROM
                                                                    maga-bigdata.maga_onboarding.distribution_center dc
                                                                JOIN
                                                                    maga-bigdata.maga_onboarding.handling_time ht ON ht.id = dc.handling_time_id
                                                                JOIN
                                                                    maga-bigdata.maga_onboarding.origin_address oa ON oa.id = dc.address_id
                                                                JOIN
                                                                    maga-bigdata.maga_onboarding.distribution_center_organizations dco ON dco.distributioncenter_id = dc.id
                                                                JOIN
                                                                    maga-bigdata.maga_onboarding.organization o ON o.id = dco.organization_id
                                                                    where name='netshoes'
                                                                    group by seller_name,default_handling_time, name) as d on d.seller_name = shp.seller_id

                                                        
                                                            where
                                                            cast(first_date_payment as date) >= '2022-04-01'
                                                            --EXTRACT(YEAR FROM date(expected_delivery_date)) = 2022
                                                            --and EXTRACT(MONTH FROM date(expected_delivery_date)) = 06
                                                            --and date(expected_delivery_date) <= current_date() 
                                                            and os.code not like '%T%'
                                                            and shp.source = '3P' 
                                                            and os.is_ignore_pickup = false
                                                            and NOT(os.amount >= 50000 AND osh.status = 12) 
                                                            and IFNULL(cancel_code,0) <> 100 
                                                            and osh.shipping_status_name in ('DESPACHADO', 'FATURADO', 'PRONTO PARA FATURAR')
                                                            and n.Ns_Entregas = 'NSE'
                                                            group by 
                                                            1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18""")

                                                            pedidos_ckeckin = dados.consulta_bigquery(projeto_gcp_1,"""SELECT DISTINCT
                                                            A.PEDS_COD COD_PEDIDO,
                                                            --A.PEDS_EXT_COD PEDIDO_EXTERNO,
                                                            --B.TROD_COD CODIGO_DA_TROCA,
                                                            B.TROD_STR_STA STATUS_checkin,
                                                            --C.ITET_DAT_PROENT DATA_CHECKIN
                                                            FROM `maga-bigdata.abacos_nets.aba_tcom_pedsai` A
                                                            INNER JOIN `maga-bigdata.abacos_nets.aba_ttrd_trodev` B ON A.CLIF_COD = B.CLIF_COD
                                                            INNER JOIN `maga-bigdata.abacos_nets.aba_ttrd_itetrd` C ON B.TROD_COD = C.TROD_COD
                                                            --WHERE A.PEDS_COD IN ('XXX')
                                                            WHERE B.TROD_STR_STA = 'Finalizado'	
                                                                --AND PEDS_VAL_PED = ITET_VAL_PRETOT
                                                                AND ITET_DAT_PROENT IS NOT NULL
                                                                AND ITET_DAT_PROENT >='2019-01-01'
                                                                --AND B.TROD_DAT_PRAPRO >= '2021-09-01'
                                                                --and A.PEDS_COD = 863085411
                                                            group by A.PEDS_COD, B.TROD_STR_STA""")

pedidos_ckeckin['COD_PEDIDO'] = pedidos_ckeckin['COD_PEDIDO'].astype(str)
base_analise_checkin = pd.merge(base_analise, pedidos_ckeckin, how='left', on='COD_PEDIDO')
base_analise_checkin_v2 = base_analise_checkin.query('STATUS_checkin!="Finalizado"')
base_analise_checkin_v2.to_csv('/content/drive/MyDrive/dados_pedidos/Nets/Ciclo_pedido/pedidos_analise.csv')'''